**Installing Packages**

In [ ]:
%pip install --quiet --upgrade discord.py

In [ ]:
%pip install --quiet --upgrade transformers accelerate torch

**Importing Utilities**

In [ ]:
from google.colab import userdata
from huggingface_hub import login
import os, gc, re, discord, asyncio, random

# HuggingFace Token
hf_token = userdata.get('HF_Read_Token')

# Discord Tokens
app_id = userdata.get("APP_ID")
public_key = userdata.get("Public_KEY")
discord_token = userdata.get("Discord_SECRET_KEY")
channel_id = int(userdata.get("Channel_ID"))

login(token=hf_token)

In [ ]:
from copy import copy

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline

Initializing Variables and Function

In [ ]:
def clear_cache():
  gc.collect()
  torch.cuda.empty_cache()
  print("\n\nCleared GPU Cache")

In [ ]:
model_name: str = "microsoft/Phi-3.5-mini-instruct"

is_cuda: bool = torch.cuda.is_available()
device = torch.device("cuda" if is_cuda else "cpu")

print("Using device:", device)
print("Using model:", model_name)

Pretraining the AI Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, use_fast=True)

print("Initialized Tokenizer")


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    device_map="auto",
    dtype=torch.float16
)

print("Pretrained the Model")

clear_cache()

**Code Starts Here**

In [ ]:
def generate_code(prompt: str, max_tokens: int=200, min_tokens: int=10) -> str:
  tokenized_prompt = tokenizer(
      prompt,
      return_tensors="pt"
  ).to(device)

  output_tokens = model.generate(
          **tokenized_prompt,
          do_sample=False,
          # temperature=0.7,
          # top_p=0.95,
          pad_token_id=tokenizer.eos_token_id,
          eos_token_id=tokenizer.eos_token_id,
          max_new_tokens=max_tokens,
          min_new_tokens=min_tokens
  )

  outputs = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

  outputs = outputs[len(prompt):]

  triple_backtick_index: int = outputs.find("```")
  if triple_backtick_index >= 0:
    outputs = outputs[:triple_backtick_index]

  return outputs

In [ ]:
intros: list[str] = [
    "Alright bro! Here it is:",
    "Got you covered, check this out:",
    "Here’s what I cooked up for you:",
    "No worries, try this:",
    "Okay, this should do the trick:",
    "Here’s a clean solution:",
    "Check this out, bro:",
    "Done! Here’s the code:"
]

outros: list[str] = [
    "Hope that helps!",
    "Let me know if you need tweaks.",
    "That should work fine.",
    "Give it a shot and tell me how it goes.",
    "You can build on this if needed.",
    "That’s one way to do it!",
    "Try running this and see.",
    "Hope this clears things up!"
]

greetings: list[str] = [
    "Yo broskis, I'm online now 🚀",
    "Guess who just woke up? 😎",
    "Your friendly neighborhood bot has arrived 🤖",
    "I'm live and kicking, let's code! 💻",
    "Reporting for duty, boss! 🫡",
    "Back online, ready to roll 🔥",
    "Bot’s in da house 🎉",
    "Systems online. Let’s get to work. ⚡",
    "Wassup fam, I'm here 🙌",
    "Ready when you are! 👊"
]

goodbyes: list[str] = [
    "Shutting down, catch you later broskis 👋",
    "Bot out! 💨",
    "Logging off, stay awesome fam ✌️",
    "Powering down... see you soon 🔌",
    "That’s all for now, peace out 🤖",
    "I’m off, don’t miss me too much 😏",
    "Goodbye world 🌍",
    "Mission complete, going offline 🫡",
    "Disconnecting... until next time ⏳",
    "Later gators 🐊"
]

In [ ]:
clear_cache()

class MyClient(discord.Client):
    max_tokens: int = 300

    async def on_ready(self):
        print(f'Logged on as {self.user}!')
        channel = self.get_channel(channel_id)
        print(channel_id, channel, sep="\n")
        if channel:
          print("Channel Found")
          await channel.send("Yo bro, I'm online and ready to code! 🚀")

    async def on_message(self, message):
        prompt: str = re.sub(r"<[@#&][0-9]+>", "", message.content).strip()
        print(f'Message from {message.author}: {prompt}')

        if message.author == self.user or not self.user in message.mentions:
            return

        Channel = message.channel
        exit_post_execution: bool = False

        # Processing exit prompt
        if re.search(r"^exit", prompt, re.IGNORECASE):
            await Channel.send(f"{random.choice(goodbyes)}")
            await Channel.send("Exiting...")
            await self.close()
            return

        if re.search(r"\nexit$", prompt, re.IGNORECASE):
            exit_post_execution = True
            prompt = re.sub(r"\nexit$", "", prompt, flags=re.IGNORECASE).strip()

        # Matching for tokens
        pre_matching = re.search(r"^\s*tokens?\s*=\s*(\d+)\s*\b", prompt, re.IGNORECASE)
        post_matching = re.search(r"\ntokens?\s*=\s*(\d+)\s*\b$", prompt, re.IGNORECASE)
        print("pre_matching=", pre_matching, sep="")
        print("post_matching=", post_matching, sep="")

        # Setting max_tokens as per user input
        if pre_matching:
            self.max_tokens = int(pre_matching.group(1))
            print(f"max_new_tokens reset to {self.max_tokens}")
            prompt = prompt.replace(pre_matching.group(0), "").strip()
            await Channel.send(f"max_new_tokens reset to {self.max_tokens}")

        if post_matching:
          prompt = prompt.replace(post_matching.group(0), "").strip()

        if prompt.strip():
            print("Generating on prompt:", prompt)
            async with Channel.typing():
              code = await asyncio.to_thread(generate_code, prompt, self.max_tokens)
            await Channel.send(f"{random.choice(intros)}:\n```\n{code}\n```\n{random.choice(outros)}")
        elif not pre_matching and not post_matching:
            await Channel.send("Yo bro, gimme a valid input")

        # Setting tokens if set at end
        if post_matching:
            self.max_tokens = int(post_matching.group(1))
            print(f"max_new_tokens reset to {self.max_tokens}")
            await Channel.send(f"max_new_tokens reset to {self.max_tokens}")

        # Exiting if exit at end
        if exit_post_execution:
            await Channel.send(f"{random.choice(goodbyes)}")
            await Channel.send("Exiting...")
            await self.close()


intents = discord.Intents.default()
intents.message_content = True
intents.guilds = True  # needed for guild/channel info
intents.members = True # optional, but often useful

client = MyClient(intents=intents)
await client.start(discord_token)